<a href="https://colab.research.google.com/github/tasninkhanlamha/SkillMorph/blob/main/Assignment7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load Dataset

```
We will build a Model to detect thread of Blockchain consensus algorithm!





In [2]:
# Imports and Setup
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend
import matplotlib.pyplot as plt
plt.switch_backend('Agg')
%matplotlib inline

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.inspection import permutation_importance

sns.set(style='whitegrid')

print('Libraries imported and setup complete.')

Libraries imported and setup complete.


Libraries imported and setup complete.

In [3]:
# Load the dataset from the local directory
df = pd.read_csv('/content/drive/MyDrive/Skill Morph/iot_blockchain_security_dataset.csv', encoding='ascii', delimiter=',')
print('Dataset loaded with shape:', df.shape)
df.head()

Dataset loaded with shape: (1000, 11)


,Device ID,IoT Layer,Request Type,Data Size (KB),Processing Time (ms),Security Threat Type,Attack Severity (0-10),Blockchain Transaction Time (ms),Consensus Mechanism,Energy Consumption (mJ),Threat Mitigated
0,D0001,Application,Data Transmission,552,42,DDoS,1,142,PBFT,1.26,1
1,D0002,Application,Data Transmission,704,24,DDoS,1,197,PoS,0.66,0
2,D0003,Device,Authentication,485,35,Eavesdropping,7,272,PoS,0.92,0
3,D0004,Application,Data Transmission,965,21,Eavesdropping,1,187,PoS,1.88,0
4,D0005,Device,Encrypted Data Transfer,653,17,Tampering,1,155,PoS,1.80,1


Data Cleaning and Preprocessing¶
Let's quickly check the data for missing values and proper data types. Note that the column names carry some extra characters (e.g., units) so be mindful when referencing them.

In [4]:
# Check for missing values and data types
print('Missing values per column:')
print(df.isnull().sum())

print('\nData types:')
print(df.dtypes)

# If any cleaning is required (e.g., trimming whitespace or renaming columns), do it here
# For example, ensuring column names are stripped:
df.columns = [col.strip() for col in df.columns]# Verify changes
print('\nCleaned column names:')
print(df.columns.tolist())

Missing values per column:
Device ID                           0
IoT Layer                           0
Request Type                        0
Data Size (KB)                      0
Processing Time (ms)                0
Security Threat Type                0
Attack Severity (0-10)              0
Blockchain Transaction Time (ms)    0
Consensus Mechanism                 0
Energy Consumption (mJ)             0
Threat Mitigated                    0
dtype: int64

Data types:
Device ID                            object
IoT Layer                            object
Request Type                         object
Data Size (KB)                        int64
Processing Time (ms)                  int64
Security Threat Type                 object
Attack Severity (0-10)                int64
Blockchain Transaction Time (ms)      int64
Consensus Mechanism                  object
Energy Consumption (mJ)             float64
Threat Mitigated                      int64
dtype: object

Cleaned column names:
['Device